In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 32.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 96.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.3 MB/s eta 0:00:00


In [6]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 7.7 MB/s eta 0:00:00:00:01


In [10]:
# Enhanced Spotify Music Recommendation System with API Integration
# Advanced ML algorithms, real-time Spotify integration, and improved UI

import pandas as pd
import numpy as np
import gradio as gr
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import requests
import random
import pickle
import os
from typing import List, Dict, Tuple, Optional
import warnings
import json
import time
from datetime import datetime
import threading
from collections import defaultdict, Counter
import re
warnings.filterwarnings('ignore')

class EnhancedSpotifyRecommendationSystem:
    def __init__(self, spotify_client_id: str = None, spotify_client_secret: str = None):
        self.df = None
        self.feature_columns = [
            'danceability', 'energy', 'key', 'loudness', 'mode',
            'speechiness', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
        ]
        
        # Enhanced preprocessing
        self.scaler = StandardScaler()
        self.minmax_scaler = MinMaxScaler()
        self.similarity_matrix = None
        self.pca = None
        self.kmeans = None
        self.genre_encoder = None
        
        # User modeling
        self.user_history = []
        self.user_preferences = None
        self.user_clusters = None
        self.listening_context = defaultdict(list)  # time-based preferences
        self.skip_patterns = []
        self.user_genre_preferences = Counter()
        
        # Advanced features
        self.features_scaled = None
        self.features_normalized = None
        self.available_features = []
        self.track_embeddings = None
        self.genre_similarity_matrix = None
        
        # Spotify API setup
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        self.spotify = None
        self.spotify_cache = {}
        
        # Recommendation engines
        self.content_filter = None
        self.collaborative_filter = None
        self.hybrid_weights = {'content': 0.6, 'collaborative': 0.3, 'popularity': 0.1}
        
        # Performance tracking
        self.recommendation_feedback = defaultdict(list)
        self.model_performance = {'accuracy': 0.0, 'diversity': 0.0, 'novelty': 0.0}
        
    def initialize_spotify_api(self, client_id: str, client_secret: str, 
                         redirect_uri: str = "http://localhost:8888/callback"):
        """Initialize Spotify API with enhanced authentication - FIXED VERSION"""
        try:
            if not client_id or not client_secret:
                print("❌ Client ID and Client Secret are required")
                return False
                
            self.spotify_client_id = client_id
            self.spotify_client_secret = client_secret
            
            # Use client credentials for general API access
            client_credentials_manager = SpotifyClientCredentials(
                client_id=client_id,
                client_secret=client_secret
            )
            self.spotify = spotipy.Spotify(
                client_credentials_manager=client_credentials_manager,
                requests_timeout=10,
                retries=3
            )
            
            # Test the connection with a simple search
            try:
                test = self.spotify.search(q='test', type='track', limit=1)
                print("✅ Spotify API initialized successfully!")
                return True
            except spotipy.exceptions.SpotifyException as e:
                if e.http_status == 401:
                    print("❌ Invalid Spotify credentials. Please check your Client ID and Secret.")
                elif e.http_status == 403:
                    print("❌ Spotify API access forbidden. Check your app settings and quotas.")
                else:
                    print(f"❌ Spotify API error: {e}")
                return False
                
        except Exception as e:
            print(f"❌ Spotify API initialization failed: {str(e)}")
            return False
        
    def search_spotify_track(self, track_name: str, artist_name: str) -> Optional[Dict]:
        """Search for track on Spotify and return enhanced metadata - FIXED VERSION"""
        if not self.spotify:
            return None
            
        try:
            # Clean the artist name - handle list format
            if isinstance(artist_name, list):
                artist_name = artist_name[0] if artist_name else ""
            elif isinstance(artist_name, str):
                # Remove brackets and quotes if present
                artist_name = re.sub(r"[\[\]']", "", str(artist_name))
                # Take first artist if multiple
                artist_name = artist_name.split(',')[0].strip()
            
            # Clean track name
            track_name = str(track_name).strip()
            artist_name = str(artist_name).strip()
            
            # Create search query - simplified approach
            query = f'"{track_name}" artist:"{artist_name}"'
            results = self.spotify.search(q=query, type='track', limit=5)
            
            if not results['tracks']['items']:
                # Try alternative search without quotes
                query = f"{track_name} {artist_name}"
                results = self.spotify.search(q=query, type='track', limit=5)
            
            if results['tracks']['items']:
                track = results['tracks']['items'][0]
                track_id = track['id']
                
                # Get audio features with proper error handling
                audio_features = None
                try:
                    if track_id:
                        # Use correct audio_features call
                        features_response = self.spotify.audio_features([track_id])
                        if features_response and features_response[0]:
                            audio_features = features_response[0]
                except Exception as e:
                    print(f"Could not get audio features for {track_name}: {str(e)}")
                    audio_features = None
                
                # Get additional track info
                track_info = {
                    'spotify_id': track_id,
                    'spotify_url': track['external_urls']['spotify'],
                    'preview_url': track.get('preview_url'),
                    'popularity': track.get('popularity', 0),
                    'album_image': track['album']['images'][0]['url'] if track['album'].get('images') else None,
                    'release_date': track['album'].get('release_date', 'Unknown'),
                    'genres': [],  # Will be filled from artist info
                    'audio_features': audio_features
                }
                
                # Get artist genres with error handling
                try:
                    if track['artists'] and len(track['artists']) > 0:
                        artist_id = track['artists'][0]['id']
                        if artist_id:
                            artist_info = self.spotify.artist(artist_id)
                            track_info['genres'] = artist_info.get('genres', [])
                except Exception as e:
                    print(f"Could not get artist info for {artist_name}: {str(e)}")
                    track_info['genres'] = []
                
                return track_info
                
        except Exception as e:
            print(f"Error searching Spotify for '{track_name}' by '{artist_name}': {str(e)}")
            return None    
    def load_data(self, file_path: str):
        """Enhanced data loading with Spotify integration"""
        print("Loading dataset...")
        self.df = pd.read_csv(file_path)
        
        # Check available features
        self.available_features = [col for col in self.feature_columns if col in self.df.columns]
        print(f"Available features: {self.available_features}")
        
        if len(self.available_features) == 0:
            raise ValueError("No audio features found in the dataset!")
        
        # Clean and standardize data
        self.df = self.df.dropna(subset=self.available_features)
        self.df = self.df.reset_index(drop=True)
        
        # Standardize column names
        column_mapping = {
            'track_name': 'name',
            'artist_name': 'artists',
            'song': 'name',
            'artist': 'artists'
        }
        
        for old_col, new_col in column_mapping.items():
            if old_col in self.df.columns and new_col not in self.df.columns:
                self.df[new_col] = self.df[old_col]
        
        # Add enhanced features
        self._add_derived_features()
        
        print(f"Dataset loaded: {len(self.df)} tracks with {len(self.available_features)} features")
        return self.df
    
    def _add_derived_features(self):
        """Add derived features for better recommendations"""
        if 'energy' in self.available_features and 'valence' in self.available_features:
            self.df['mood_energy'] = self.df['energy'] * self.df['valence']
            self.available_features.append('mood_energy')
        
        if 'danceability' in self.available_features and 'tempo' in self.available_features:
            self.df['dance_tempo'] = self.df['danceability'] * (self.df['tempo'] / 200)
            self.available_features.append('dance_tempo')
        
        if 'acousticness' in self.available_features and 'instrumentalness' in self.available_features:
            self.df['acoustic_instrumental'] = (self.df['acousticness'] + self.df['instrumentalness']) / 2
            self.available_features.append('acoustic_instrumental')
        
        # Add popularity score if not available
        if 'popularity' not in self.df.columns:
            self.df['popularity'] = self._calculate_popularity_score()
    
    def _calculate_popularity_score(self) -> pd.Series:
        """Calculate popularity score from available features"""
        popularity = pd.Series(0.0, index=self.df.index)
        
        weights = {
            'danceability': 0.25,
            'energy': 0.25,
            'valence': 0.2,
            'acousticness': -0.15,  # Less acoustic = more popular
            'speechiness': -0.1,    # Less speech = more popular
            'liveness': -0.05       # Less live = more popular
        }
        
        for feature, weight in weights.items():
            if feature in self.available_features:
                if weight < 0:
                    popularity += (1 - self.df[feature]) * abs(weight)
                else:
                    popularity += self.df[feature] * weight
        
        # Normalize to 0-100 scale
        popularity = (popularity - popularity.min()) / (popularity.max() - popularity.min()) * 100
        return popularity
    
    def preprocess_features(self):
        """Enhanced feature preprocessing with multiple scaling methods"""
        print("Preprocessing features...")
        
        # Scale features using different methods
        features = self.df[self.available_features].values
        self.features_scaled = self.scaler.fit_transform(features)
        self.features_normalized = self.minmax_scaler.fit_transform(features)
        
        # Apply PCA for dimensionality reduction
        n_features = len(self.available_features)
        n_samples = len(self.df)
        
        if n_features > 5:  # Only apply PCA if we have enough features
            n_components = min(max(5, n_features // 2), 50)
            self.pca = PCA(n_components=n_components)
            self.features_scaled = self.pca.fit_transform(self.features_scaled)
            print(f"PCA applied: reduced from {n_features} to {n_components} dimensions")
        
        # Apply clustering for user segmentation
        n_clusters = min(20, max(5, len(self.df) // 1000))
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.df['cluster'] = self.kmeans.fit_predict(self.features_scaled)
        
        print("Enhanced preprocessing completed")
    
    def build_similarity_matrix(self, sample_size: int = 50000):
        """Build enhanced similarity matrix with multiple similarity measures"""
        print(f"Building enhanced similarity matrix...")
        
        # Sample for memory efficiency
        if len(self.df) > sample_size:
            sample_indices = np.random.choice(len(self.df), sample_size, replace=False)
            sample_features = self.features_scaled[sample_indices]
            self.sample_indices = sample_indices
        else:
            sample_features = self.features_scaled
            self.sample_indices = np.arange(len(self.df))
        
        # Compute multiple similarity matrices
        cosine_sim = cosine_similarity(sample_features)
        
        # Add genre similarity if available
        if 'genres' in self.df.columns:
            genre_sim = self._compute_genre_similarity()
            # Combine similarities
            self.similarity_matrix = 0.7 * cosine_sim + 0.3 * genre_sim
        else:
            self.similarity_matrix = cosine_sim
        
        print("Enhanced similarity matrix built")
    
    def _compute_genre_similarity(self) -> np.ndarray:
        """Compute genre-based similarity matrix"""
        # This is a simplified version - in practice, you'd use more sophisticated genre embeddings
        n_samples = len(self.sample_indices)
        genre_sim = np.eye(n_samples)  # Identity matrix as placeholder
        return genre_sim
    
    def get_hybrid_recommendations(self, track_index: int = None, n_recommendations: int = 10,
                                 context: str = 'general') -> List[int]:
        """Advanced hybrid recommendation system"""
        recommendations = []
        
        # Content-based recommendations
        if track_index is not None:
            content_recs = self._get_content_based_recommendations(track_index, n_recommendations)
            recommendations.extend([(idx, 'content') for idx in content_recs])
        
        # Collaborative filtering recommendations
        if self.user_preferences is not None:
            collab_recs = self._get_collaborative_recommendations(n_recommendations)
            recommendations.extend([(idx, 'collaborative') for idx in collab_recs])
        
        # Context-aware recommendations
        context_recs = self._get_context_aware_recommendations(context, n_recommendations)
        recommendations.extend([(idx, 'context') for idx in context_recs])
        
        # Popularity-based recommendations
        popular_recs = self._get_popularity_recommendations(n_recommendations)
        recommendations.extend([(idx, 'popularity') for idx in popular_recs])
        
        # Combine and rank recommendations
        final_recs = self._rank_hybrid_recommendations(recommendations, n_recommendations)
        
        return final_recs
    
    def _get_content_based_recommendations(self, track_index: int, n_recommendations: int) -> List[int]:
        """Enhanced content-based filtering"""
        if self.similarity_matrix is None:
            return []
        
        if track_index not in self.sample_indices:
            # Find similar tracks using feature similarity
            track_features = self.features_scaled[track_index].reshape(1, -1)
            sample_features = self.features_scaled[self.sample_indices]
            similarities = cosine_similarity(track_features, sample_features)[0]
            similar_indices = np.argsort(similarities)[::-1][1:n_recommendations+1]
            return [self.sample_indices[i] for i in similar_indices]
        else:
            sample_pos = np.where(self.sample_indices == track_index)[0][0]
            similarities = self.similarity_matrix[sample_pos]
            similar_indices = np.argsort(similarities)[::-1][1:n_recommendations+1]
            return [self.sample_indices[i] for i in similar_indices]
    
    def _get_collaborative_recommendations(self, n_recommendations: int) -> List[int]:
        """Collaborative filtering based on user clusters"""
        if self.user_preferences is None:
            return []
        
        # Find similar users based on listening history
        user_cluster = self.kmeans.predict(self.user_preferences.reshape(1, -1))[0]
        similar_users_tracks = self.df[self.df['cluster'] == user_cluster].index.tolist()
        
        # Filter out already heard tracks
        seen_tracks = set(self.user_history)
        recommendations = [idx for idx in similar_users_tracks if idx not in seen_tracks]
        
        return recommendations[:n_recommendations]
    
    def _get_context_aware_recommendations(self, context: str, n_recommendations: int) -> List[int]:
        """Context-aware recommendations based on time, mood, activity"""
        context_filters = {
            'workout': {'energy': (0.7, 1.0), 'tempo': (120, 200)},
            'chill': {'energy': (0.0, 0.5), 'valence': (0.3, 0.8)},
            'party': {'danceability': (0.7, 1.0), 'energy': (0.7, 1.0)},
            'focus': {'instrumentalness': (0.5, 1.0), 'speechiness': (0.0, 0.3)},
            'sleep': {'energy': (0.0, 0.3), 'tempo': (60, 100)},
            'general': {}  # No specific filters
        }
        
        filters = context_filters.get(context, {})
        mask = pd.Series(True, index=self.df.index)
        
        for feature, (min_val, max_val) in filters.items():
            if feature in self.available_features:
                mask &= (self.df[feature] >= min_val) & (self.df[feature] <= max_val)
        
        filtered_indices = self.df[mask].index.tolist()
        return random.sample(filtered_indices, min(n_recommendations, len(filtered_indices)))
    
    def _get_popularity_recommendations(self, n_recommendations: int) -> List[int]:
        """Enhanced popularity-based recommendations"""
        if 'popularity' in self.df.columns:
            popular_tracks = self.df.nlargest(n_recommendations * 2, 'popularity')
        else:
            # Use derived popularity score
            popularity_score = self._calculate_popularity_score()
            popular_tracks = self.df.loc[popularity_score.nlargest(n_recommendations * 2).index]
        
        return random.sample(popular_tracks.index.tolist(), 
                           min(n_recommendations, len(popular_tracks)))
    
    def _rank_hybrid_recommendations(self, recommendations: List[Tuple], n_recommendations: int) -> List[int]:
        """Rank and combine recommendations from different sources"""
        # Count recommendations and apply weights
        rec_scores = defaultdict(float)
        rec_sources = defaultdict(list)
        
        for track_idx, source in recommendations:
            weight = self.hybrid_weights.get(source, 0.1)
            rec_scores[track_idx] += weight
            rec_sources[track_idx].append(source)
        
        # Boost tracks recommended by multiple sources
        for track_idx, sources in rec_sources.items():
            if len(sources) > 1:
                rec_scores[track_idx] *= 1.2
        
        # Sort by score and remove duplicates
        sorted_recs = sorted(rec_scores.items(), key=lambda x: x[1], reverse=True)
        final_recs = [track_idx for track_idx, _ in sorted_recs[:n_recommendations]]
        
        return final_recs
    
    def get_infinite_feed_recommendations(self, n_recommendations: int = 20) -> List[Dict]:
        """Generate infinite feed with Spotify integration"""
        # Get hybrid recommendations
        if self.user_history:
            recent_track = self.user_history[-1]
            recommendations = self.get_hybrid_recommendations(recent_track, n_recommendations)
        else:
            recommendations = self.get_hybrid_recommendations(n_recommendations=n_recommendations)
        
        # Format with Spotify integration
        formatted_recommendations = []
        for idx in recommendations[:n_recommendations]:
            try:
                track_info = self._format_track_with_spotify(idx)
                if track_info:
                    formatted_recommendations.append(track_info)
            except Exception as e:
                print(f"Error formatting track {idx}: {e}")
                continue
        
        return formatted_recommendations
    
    def _format_track_with_spotify(self, idx: int) -> Optional[Dict]:
        """Format track information with corrected Spotify integration - FIXED VERSION"""
        try:
            row = self.df.iloc[idx]
            
            # Clean artist name handling
            artist_name = row['artists']
            if isinstance(artist_name, str):
                # Handle string representation of lists
                if artist_name.startswith('[') and artist_name.endswith(']'):
                    artist_name = artist_name.strip('[]').replace("'", "").replace('"', '')
                    artist_name = artist_name.split(',')[0].strip()
            elif isinstance(artist_name, list):
                artist_name = artist_name[0] if artist_name else 'Unknown Artist'
            
            # Basic track info
            track_info = {
                'index': idx,
                'track_name': str(row['name']).strip(),
                'artist_name': str(artist_name).strip(),
                'album': str(row.get('album', 'Unknown')),
                'year': int(row['year']) if 'year' in row and pd.notna(row['year']) else 'Unknown',
                'explicit': bool(row.get('explicit', False)),
                'cluster': int(row.get('cluster', 0)),
            }
            
            # Add audio features safely
            for feature in ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'speechiness']:
                if feature in self.available_features:
                    try:
                        value = float(row[feature])
                        track_info[feature] = round(value, 3) if not pd.isna(value) else 'N/A'
                    except (ValueError, TypeError):
                        track_info[feature] = 'N/A'
                else:
                    track_info[feature] = 'N/A'
            
            # Add Spotify integration with better error handling
            if self.spotify:
                try:
                    spotify_info = self.search_spotify_track(
                        track_info['track_name'], 
                        track_info['artist_name']
                    )
                    if spotify_info:
                        track_info.update({
                            'spotify_url': spotify_info['spotify_url'],
                            'preview_url': spotify_info['preview_url'],
                            'album_image': spotify_info['album_image'],
                            'spotify_popularity': spotify_info['popularity'],
                            'genres': spotify_info['genres'],
                            'spotify_id': spotify_info['spotify_id']
                        })
                    else:
                        # Add fallback values
                        track_info.update({
                            'spotify_url': None,
                            'preview_url': None,
                            'album_image': None,
                            'spotify_popularity': 0,
                            'genres': [],
                            'spotify_id': None
                        })
                except Exception as e:
                    print(f"Spotify integration error for track {idx}: {str(e)}")
                    # Add fallback values
                    track_info.update({
                        'spotify_url': None,
                        'preview_url': None,
                        'album_image': None,
                        'spotify_popularity': 0,
                        'genres': [],
                        'spotify_id': None
                    })
            
            return track_info
            
        except Exception as e:
            print(f"Error formatting track {idx}: {e}")
            return None 
    def update_user_preferences(self, liked_tracks: List[int], disliked_tracks: List[int] = None,
                              context: str = 'general'):
        """Enhanced user preference learning"""
        if not liked_tracks:
            return
        
        # Update basic preferences
        liked_features = self.features_scaled[liked_tracks]
        self.user_preferences = np.mean(liked_features, axis=0)
        
        # Update context preferences
        current_time = datetime.now().hour
        self.listening_context[current_time].extend(liked_tracks)
        
        # Update genre preferences if available
        for track_idx in liked_tracks:
            if 'genres' in self.df.columns:
                genres = str(self.df.iloc[track_idx].get('genres', '')).split(',')
                for genre in genres:
                    if genre.strip():
                        self.user_genre_preferences[genre.strip()] += 1
        
        # Learn from dislikes
        if disliked_tracks:
            disliked_features = self.features_scaled[disliked_tracks]
            disliked_avg = np.mean(disliked_features, axis=0)
            # Move preferences away from disliked features
            self.user_preferences = self.user_preferences - 0.2 * disliked_avg
            self.skip_patterns.extend(disliked_tracks)
        
        # Update hybrid weights based on user feedback
        self._update_hybrid_weights()
    
    def _update_hybrid_weights(self):
        """Dynamically adjust hybrid weights based on user feedback"""
        if len(self.recommendation_feedback) > 10:
            # Analyze which recommendation sources perform better
            source_performance = defaultdict(list)
            for track_idx, feedback in self.recommendation_feedback.items():
                for fb in feedback:
                    source_performance[fb['source']].append(fb['rating'])
            
            # Update weights based on performance
            total_weight = 1.0
            for source in ['content', 'collaborative', 'popularity']:
                if source in source_performance:
                    avg_rating = np.mean(source_performance[source])
                    self.hybrid_weights[source] = max(0.1, avg_rating / 5.0)
            
            # Normalize weights
            weight_sum = sum(self.hybrid_weights.values())
            for source in self.hybrid_weights:
                self.hybrid_weights[source] /= weight_sum
    
    def like_track(self, track_index: int, rating: float = 5.0, context: str = 'general'):
        """Enhanced track liking with ratings and context"""
        if track_index not in self.user_history:
            self.user_history.append(track_index)
        
        # Store feedback
        self.recommendation_feedback[track_index].append({
            'rating': rating,
            'context': context,
            'timestamp': datetime.now(),
            'source': 'user_action'
        })
        
        # Update preferences with context
        self.update_user_preferences(self.user_history[-10:], context=context)
    
    def get_playlist_recommendations(self, playlist_name: str, n_tracks: int = 20,
                                   context: str = 'general') -> List[Dict]:
        """Generate themed playlist recommendations"""
        playlist_themes = {
            'workout': self._get_context_aware_recommendations('workout', n_tracks),
            'chill': self._get_context_aware_recommendations('chill', n_tracks),
            'party': self._get_context_aware_recommendations('party', n_tracks),
            'focus': self._get_context_aware_recommendations('focus', n_tracks),
            'discovery': self._get_discovery_recommendations(n_tracks),
            'throwback': self._get_throwback_recommendations(n_tracks)
        }
        
        track_indices = playlist_themes.get(playlist_name, 
                                          self.get_infinite_feed_recommendations(n_tracks))
        
        if isinstance(track_indices[0], dict):
            return track_indices
        else:
            return [self._format_track_with_spotify(idx) for idx in track_indices 
                   if self._format_track_with_spotify(idx) is not None]
    
    def _get_discovery_recommendations(self, n_tracks: int) -> List[int]:
        """Get recommendations for music discovery (less popular, diverse tracks)"""
        if 'popularity' in self.df.columns:
            # Focus on less popular but quality tracks
            mid_popularity = self.df[(self.df['popularity'] > 30) & (self.df['popularity'] < 70)]
        else:
            # Use random sampling for discovery
            mid_popularity = self.df.sample(min(n_tracks * 3, len(self.df)))
        
        return random.sample(mid_popularity.index.tolist(), 
                           min(n_tracks, len(mid_popularity)))
    
    def _get_throwback_recommendations(self, n_tracks: int) -> List[int]:
        """Get recommendations for older tracks"""
        if 'year' in self.df.columns:
            current_year = datetime.now().year
            throwback_tracks = self.df[self.df['year'] <= current_year - 5]
        else:
            throwback_tracks = self.df.sample(min(n_tracks * 2, len(self.df)))
        
        return random.sample(throwback_tracks.index.tolist(),
                           min(n_tracks, len(throwback_tracks)))
    
    def search_tracks(self, query: str, limit: int = 10, search_spotify: bool = True) -> List[Dict]:
        """Enhanced search with Spotify integration"""
        query = query.lower()
        
        # Search in local dataset
        mask = (
            self.df['name'].str.lower().str.contains(query, na=False) |
            self.df['artists'].str.lower().str.contains(query, na=False)
        )
        
        if 'album' in self.df.columns:
            mask = mask | self.df['album'].str.lower().str.contains(query, na=False)
        
        local_results = self.df[mask].head(limit)
        
        # Format results with Spotify integration
        formatted_results = []
        for idx, row in local_results.iterrows():
            track_info = self._format_track_with_spotify(idx)
            if track_info:
                formatted_results.append(track_info)
        
        return formatted_results
    
    def get_user_insights(self) -> Dict:
        """Get comprehensive user insights and analytics"""
        if not self.user_history:
            return {"message": "No listening history available"}
        
        insights = {
            'total_tracks_liked': len(self.user_history),
            'favorite_genres': dict(self.user_genre_preferences.most_common(5)),
            'listening_patterns': self._analyze_listening_patterns(),
            'music_taste_profile': self._get_taste_profile(),
            'recommendation_accuracy': self._calculate_recommendation_accuracy(),
            'diversity_score': self._calculate_diversity_score(),
            'discovery_rate': self._calculate_discovery_rate()
        }
        
        return insights
    
    def _analyze_listening_patterns(self) -> Dict:
        """Analyze user's listening patterns by time"""
        patterns = {}
        for hour, tracks in self.listening_context.items():
            if tracks:
                avg_energy = np.mean([self.df.iloc[idx].get('energy', 0.5) for idx in tracks])
                avg_valence = np.mean([self.df.iloc[idx].get('valence', 0.5) for idx in tracks])
                patterns[f"{hour}:00"] = {
                    'energy': round(avg_energy, 3),
                    'mood': round(avg_valence, 3),
                    'track_count': len(tracks)
                }
        return patterns
    
    def _get_taste_profile(self) -> Dict:
        """Get detailed music taste profile"""
        if not self.user_history:
            return {}
        
        liked_tracks = self.df.iloc[self.user_history]
        profile = {}
        
        for feature in self.available_features:
            if feature in liked_tracks.columns:
                avg_value = liked_tracks[feature].mean()
                profile[feature] = {
                    'average': round(avg_value, 3),
                    'preference': self._interpret_feature_preference(feature, avg_value)
                }
        
        return profile
    
    def _interpret_feature_preference(self, feature: str, value: float) -> str:
        """Interpret feature values into readable preferences"""
        interpretations = {
            'danceability': ['Not danceable', 'Somewhat danceable', 'Very danceable'],
            'energy': ['Low energy', 'Moderate energy', 'High energy'],
            'valence': ['Sad/Dark', 'Neutral mood', 'Happy/Uplifting'],
            'acousticness': ['Electronic', 'Mixed', 'Acoustic'],
            'speechiness': ['Instrumental', 'Some vocals', 'Very vocal/Rap'],
            'instrumentalness': ['Vocal tracks', 'Mixed', 'Instrumental']
        }
        
        if feature in interpretations:
            if value < 0.33:
                return interpretations[feature][0]
            elif value < 0.67:
                return interpretations[feature][1]
            else:
                return interpretations[feature][2]
        
        return f"Value: {value:.3f}"
    
    def _calculate_recommendation_accuracy(self) -> float:
        """Calculate recommendation system accuracy based on user feedback"""
        if not self.recommendation_feedback:
            return 0.0
        
        total_ratings = []
        for track_feedback in self.recommendation_feedback.values():
            for feedback in track_feedback:
                total_ratings.append(feedback['rating'])
        
        return round(np.mean(total_ratings) / 5.0, 3) if total_ratings else 0.0
    
    def _calculate_diversity_score(self) -> float:
        """Calculate diversity of user's music taste"""
        if len(self.user_history) < 5:
            return 0.0
        
        # Calculate diversity based on feature variance
        liked_tracks = self.df.iloc[self.user_history]
        feature_vars = []
        
        for feature in self.available_features:
            if feature in liked_tracks.columns:
                feature_vars.append(liked_tracks[feature].var())
        
        return round(np.mean(feature_vars), 3) if feature_vars else 0.0
    
    def _calculate_discovery_rate(self) -> float:
        """Calculate how much new music user discovers"""
        if not self.user_history:
            return 0.0
        
        # Simple measure: ratio of unique artists in history
        if 'artists' in self.df.columns:
            liked_tracks = self.df.iloc[self.user_history]
            unique_artists = liked_tracks['artists'].nunique()
            total_tracks = len(self.user_history)
            return round(unique_artists / total_tracks, 3)
        
        return 0.0

# Global variables for Gradio interface
recommender = EnhancedSpotifyRecommendationSystem()
current_recommendations = []
user_liked_tracks = []
spotify_credentials = {'client_id': '', 'client_secret': ''}

def initialize_system_with_spotify(dataset_path: str, spotify_client_id: str = "", 
                                 spotify_client_secret: str = ""):
    """Initialize the enhanced recommendation system"""
    global recommender, spotify_credentials
    
    try:
        # Store Spotify credentials
        if spotify_client_id and spotify_client_secret:
            spotify_credentials['client_id'] = spotify_client_id
            spotify_credentials['client_secret'] = spotify_client_secret
            
            # Initialize Spotify API
            if recommender.initialize_spotify_api(spotify_client_id, spotify_client_secret):
                spotify_status = "✅ Spotify API connected successfully!"
            else:
                spotify_status = "⚠️ Spotify API connection failed. Recommendations will work without Spotify links."
        else:
            spotify_status = "⚠️ No Spotify credentials provided. Add them for full functionality."
        
        # Try multiple dataset paths
        possible_paths = [
            dataset_path,
            "/kaggle/input/spotify-12m-songs/tracks_features.csv",
            "/kaggle/input/spotify-1-2-million-songs/tracks_features.csv",
            "/content/tracks_features.csv",
            "tracks_features.csv"
        ]
        
        dataset_loaded = False
        for path in possible_paths:
            try:
                if path and os.path.exists(path):
                    print(f"Loading dataset from: {path}")
                    recommender.load_data(path)
                    recommender.preprocess_features()
                    recommender.build_similarity_matrix()
                    dataset_loaded = True
                    break
            except Exception as e:
                print(f"Failed to load from {path}: {str(e)}")
                continue
        
        if dataset_loaded:
            status_message = f"""
🎉 **Enhanced Spotify Recommendation System Initialized!**

📊 **Dataset Status:**
- Loaded {len(recommender.df)} tracks
- Available features: {len(recommender.available_features)}
- Clusters created: {recommender.kmeans.n_clusters if recommender.kmeans else 0}

🎵 **Spotify Integration:**
{spotify_status}

🤖 **AI Features Active:**
- ✅ Hybrid recommendation engine
- ✅ Context-aware recommendations  
- ✅ User preference learning
- ✅ Content-based filtering
- ✅ Collaborative filtering
- ✅ Advanced analytics

Ready to discover amazing music! 🚀
            """
            return status_message
        else:
            return "❌ Error: Could not load dataset. Please check the file path."
            
    except Exception as e:
        return f"❌ Initialization error: {str(e)}"

def get_enhanced_recommendations(context: str = "general"):
    """Get enhanced recommendations with corrected Spotify display - FIXED VERSION"""
    global recommender, current_recommendations
    
    try:
        if recommender.df is None:
            return "❌ Please initialize the system first!"
        
        current_recommendations = recommender.get_infinite_feed_recommendations(20)
        
        if not current_recommendations:
            return "No recommendations available. Please try again."
        
        # Format recommendations with corrected display - NO CLICKABLE LINKS
        formatted_output = f"🎵 **Your {context.title()} Music Recommendations:**\n\n"
        
        for i, track in enumerate(current_recommendations, 1):
            explicit_tag = " 🔞" if track.get('explicit', False) else ""
            
            # Display track title without clickable link to avoid iframe issues
            track_title = f"**{track['track_name']}**"
            
            formatted_output += f"{i}. {track_title}{explicit_tag} by *{track['artist_name']}*\n"
            formatted_output += f"   📀 Album: {track['album']} ({track['year']})\n"
            
            # Add Spotify URL as plain text that users can copy
            if track.get('spotify_url'):
                formatted_output += f"   🎵 Spotify: `{track['spotify_url']}`\n"
            
            # Show genres if available
            if track.get('genres'):
                genres_str = ', '.join(track['genres'][:3])
                formatted_output += f"   🎸 Genres: {genres_str}\n"
            
            # Audio features
            feature_info = []
            if track['danceability'] != 'N/A':
                feature_info.append(f"Dance: {track['danceability']}")
            if track['energy'] != 'N/A':
                feature_info.append(f"Energy: {track['energy']}")
            if track['valence'] != 'N/A':
                feature_info.append(f"Mood: {track['valence']}")
            
            if feature_info:
                formatted_output += f"   🎵 {' | '.join(feature_info)}\n"
            
            # Add preview URL as plain text
            if track.get('preview_url'):
                formatted_output += f"   🔊 Preview: `{track['preview_url']}`\n"
            
            formatted_output += "\n"
        
        return formatted_output
    
    except Exception as e:
        return f"Error getting recommendations: {str(e)}"
        
def like_track_with_rating(track_number: int, rating: float = 5.0, context: str = "general"):
    """Like a track with rating and context"""
    global recommender, current_recommendations, user_liked_tracks
    
    try:
        if recommender.df is None:
            return "❌ Please initialize the system first!"
        
        if not current_recommendations or track_number < 1 or track_number > len(current_recommendations):
            return "Invalid track number. Please select a valid track from the recommendations."
        
        track = current_recommendations[track_number - 1]
        track_index = track['index']
        
        # Add to user's liked tracks
        user_liked_tracks.append(track_index)
        recommender.like_track(track_index, rating, context)
        
        # Provide feedback based on rating
        rating_feedback = {
            5.0: "❤️ Loved it!",
            4.0: "👍 Really liked it!",
            3.0: "😊 It was okay",
            2.0: "😐 Not great",
            1.0: "👎 Disliked"
        }
        
        feedback = rating_feedback.get(rating, "✅ Rated")
        
        return f"""{feedback} **{track['track_name']}** by *{track['artist_name']}*

🎯 Your preferences have been updated based on this {rating}/5 rating!
🤖 The AI is learning your taste and will improve recommendations.

Total tracks liked: {len(user_liked_tracks)}
        """
    
    except Exception as e:
        return f"Error rating track: {str(e)}"

def search_enhanced_music(query: str):
    """Enhanced music search with corrected Spotify integration - FIXED VERSION"""
    global recommender
    
    try:
        if recommender.df is None:
            return "❌ Please initialize the system first!"
        
        if not query.strip():
            return "Please enter a search query."
        
        results = recommender.search_tracks(query, 15)
        
        if not results:
            return "No tracks found for your search query."
        
        formatted_output = f"🔍 **Search Results for '{query}':**\n\n"
        
        for i, track in enumerate(results, 1):
            explicit_tag = " 🔞" if track.get('explicit', False) else ""
            
            # Display track title without clickable link
            track_title = f"**{track['track_name']}**"
            
            formatted_output += f"{i}. {track_title}{explicit_tag} by *{track['artist_name']}*\n"
            formatted_output += f"   📀 {track['album']} ({track['year']})\n"
            
            # Add Spotify URL as copyable text
            if track.get('spotify_url'):
                formatted_output += f"   🎵 Spotify: `{track['spotify_url']}`\n"
            
            if track.get('genres'):
                genres_str = ', '.join(track['genres'][:2])
                formatted_output += f"   🎸 {genres_str}\n"
            
            if track.get('preview_url'):
                formatted_output += f"   🔊 Preview: `{track['preview_url']}`\n"
            
            formatted_output += "\n"
        
        return formatted_output
    
    except Exception as e:
        return f"Error searching: {str(e)}"

def load_tracks_to_player():
    """Load current recommendations into audio players"""
    global current_recommendations
    
    if not current_recommendations:
        return ["No recommendations available"] + [None] * 10
    
    outputs = []
    for i in range(5):
        if i < len(current_recommendations):
            track = current_recommendations[i]
            track_info_text = f"**{track['track_name']}** by *{track['artist_name']}*"
            preview_url = track.get('preview_url')
            
            outputs.append(track_info_text)
            outputs.append(preview_url)
        else:
            outputs.append("No track")
            outputs.append(None)
    
    return outputs

def generate_playlist(playlist_type: str = "discovery", num_tracks: int = 20):
    """Generate themed playlists - FIXED VERSION"""
    global recommender
    
    try:
        if recommender.df is None:
            return "❌ Please initialize the system first!"
        
        playlist_descriptions = {
            'workout': 'High-energy tracks perfect for exercise 💪',
            'chill': 'Relaxing tracks for unwinding 😌',
            'party': 'Upbeat tracks to get the party started 🎉',
            'focus': 'Instrumental and ambient tracks for concentration 🧠',
            'discovery': 'Hidden gems and new music discoveries 🔍',
            'throwback': 'Classic tracks from the past 📻'
        }
        
        playlist_tracks = recommender.get_playlist_recommendations(
            playlist_type, num_tracks, playlist_type
        )
        
        if not playlist_tracks:
            return f"Could not generate {playlist_type} playlist. Try again!"
        
        description = playlist_descriptions.get(playlist_type, 'Custom playlist')
        formatted_output = f"🎵 **{playlist_type.title()} Playlist** - {description}\n\n"
        
        total_duration = 0
        for i, track in enumerate(playlist_tracks, 1):
            explicit_tag = " 🔞" if track.get('explicit', False) else ""
            
            formatted_output += f"**{i}. {track['track_name']}**{explicit_tag} by *{track['artist_name']}*\n"
            
            # Add Spotify URL as copyable text instead of clickable link
            if track.get('spotify_url'):
                formatted_output += f"   🎵 Spotify: `{track['spotify_url']}`\n"
            
            # Add duration if available
            if 'duration_ms' in recommender.available_features:
                duration_ms = recommender.df.iloc[track['index']].get('duration_ms', 0)
                if duration_ms:
                    minutes = int(duration_ms / 60000)
                    seconds = int((duration_ms % 60000) / 1000)
                    formatted_output += f"   ⏱️ {minutes}:{seconds:02d}\n"
                    total_duration += duration_ms
            
            formatted_output += "\n"
        
        # Add playlist summary
        if total_duration > 0:
            total_minutes = int(total_duration / 60000)
            formatted_output += f"\n📊 **Playlist Summary:** {len(playlist_tracks)} tracks, ~{total_minutes} minutes"
        
        formatted_output += f"\n\n💡 **Tip:** Copy the Spotify URLs above and paste them in your browser to listen!"
        
        return formatted_output
    
    except Exception as e:
        return f"Error generating playlist: {str(e)}"

def get_user_analytics():
    """Get comprehensive user analytics and insights"""
    global recommender, user_liked_tracks
    
    try:
        if recommender.df is None:
            return "❌ Please initialize the system first!"
        
        insights = recommender.get_user_insights()
        
        if insights.get('message'):
            return insights['message']
        
        analytics_output = "📊 **Your Music Analytics Dashboard**\n\n"
        
        # Basic stats
        analytics_output += f"**📈 Listening Stats:**\n"
        analytics_output += f"• Total tracks liked: {insights['total_tracks_liked']}\n"
        analytics_output += f"• Recommendation accuracy: {insights['recommendation_accuracy']:.1%}\n"
        analytics_output += f"• Music diversity score: {insights['diversity_score']:.3f}\n"
        analytics_output += f"• Discovery rate: {insights['discovery_rate']:.1%}\n\n"
        
        # Favorite genres
        if insights['favorite_genres']:
            analytics_output += "**🎸 Top Genres:**\n"
            for genre, count in list(insights['favorite_genres'].items())[:5]:
                analytics_output += f"• {genre}: {count} tracks\n"
            analytics_output += "\n"
        
        # Music taste profile
        if insights['music_taste_profile']:
            analytics_output += "**🎵 Your Music Taste Profile:**\n"
            for feature, data in insights['music_taste_profile'].items():
                if isinstance(data, dict) and 'preference' in data:
                    analytics_output += f"• {feature.title()}: {data['preference']} ({data['average']:.3f})\n"
            analytics_output += "\n"
        
        # Listening patterns
        if insights['listening_patterns']:
            analytics_output += "**⏰ Listening Patterns by Hour:**\n"
            for hour, pattern in sorted(insights['listening_patterns'].items()):
                analytics_output += f"• {hour}: Energy {pattern['energy']:.2f}, Mood {pattern['mood']:.2f} ({pattern['track_count']} tracks)\n"
        
        return analytics_output
    
    except Exception as e:
        return f"Error getting analytics: {str(e)}"
        
def create_enhanced_gradio_interface():
    """Create the enhanced Gradio interface"""
    
    with gr.Blocks(title="Enhanced Spotify Music Recommendation System", 
                   theme=gr.themes.Soft()) as app:
        
        gr.Markdown("""
        # 🎵 Enhanced Spotify Music Recommendation System
        ### AI-Powered Music Discovery with Spotify Integration
        
        🚀 **New Features:**
        - Real Spotify API integration with playable links
        - Advanced hybrid recommendation algorithms
        - Context-aware recommendations (workout, chill, party, etc.)
        - Comprehensive user analytics and insights
        - Smart playlist generation
        - Enhanced user preference learning
        """)
        
        with gr.Tab("🏠 Setup & Home"):
            gr.Markdown("## 🔧 System Setup")
            
            with gr.Row():
                dataset_path = gr.Textbox(
                    label="Dataset Path", 
                    placeholder="/path/to/your/spotify_dataset.csv",
                    value=""
                )
            
            with gr.Row():
                spotify_client_id = gr.Textbox(
                    label="Spotify Client ID (Optional)", 
                    placeholder="Your Spotify App Client ID",
                    type="password"
                )
                spotify_client_secret = gr.Textbox(
                    label="Spotify Client Secret (Optional)", 
                    placeholder="Your Spotify App Client Secret",
                    type="password"
                )
            
            init_btn = gr.Button("🚀 Initialize Enhanced System", variant="primary", size="lg")
            init_output = gr.Markdown(label="System Status")
            
            init_btn.click(
                initialize_system_with_spotify,
                inputs=[dataset_path, spotify_client_id, spotify_client_secret],
                outputs=init_output
            )
            
            gr.Markdown("---")
            
            gr.Markdown("## 🎵 Get Recommendations")
            
            with gr.Row():
                context_dropdown = gr.Dropdown(
                    choices=["general", "workout", "chill", "party", "focus", "sleep"],
                    value="general",
                    label="Listening Context"
                )
                get_recs_btn = gr.Button("🔄 Get Smart Recommendations", variant="secondary")
            
            recommendations_output = gr.Markdown(label="AI Recommendations")
            
            get_recs_btn.click(
                get_enhanced_recommendations,
                inputs=context_dropdown,
                outputs=recommendations_output
            )
            
            gr.Markdown("---")
            
            gr.Markdown("## ❤️ Rate a Track")
            with gr.Row():
                track_number_input = gr.Number(label="Track Number", value=1, minimum=1, maximum=20)
                rating_slider = gr.Slider(label="Rating", minimum=1, maximum=5, value=5, step=1)
                context_rating = gr.Dropdown(
                    choices=["general", "workout", "chill", "party", "focus"],
                    value="general",
                    label="Context"
                )
            
            rate_btn = gr.Button("⭐ Rate Track", variant="primary")
            rate_output = gr.Markdown(label="Rating Status")
            
            rate_btn.click(
                like_track_with_rating,
                inputs=[track_number_input, rating_slider, context_rating],
                outputs=rate_output
            )
        
        with gr.Tab("🔍 Smart Search"):
            gr.Markdown("## 🎯 Enhanced Music Search")
            
            with gr.Row():
                search_input = gr.Textbox(
                    label="Search Music", 
                    placeholder="Enter song, artist, album, or genre...",
                    scale=3
                )
                search_btn = gr.Button("🔍 Search", variant="primary", scale=1)
            
            search_output = gr.Markdown(label="Search Results with Spotify Links")
            
            search_btn.click(search_enhanced_music, inputs=search_input, outputs=search_output)
        
        with gr.Tab("🎵 Smart Playlists"):
            gr.Markdown("## 🎼 AI-Generated Playlists")
            
            with gr.Row():
                playlist_type = gr.Dropdown(
                    choices=["workout", "chill", "party", "focus", "discovery", "throwback"],
                    value="discovery",
                    label="Playlist Type"
                )
                num_tracks = gr.Slider(
                    label="Number of Tracks",
                    minimum=10,
                    maximum=50,
                    value=20,
                    step=5
                )
            
            playlist_btn = gr.Button("🎵 Generate Playlist", variant="secondary")
            playlist_output = gr.Markdown(label="Generated Playlist")
            
            playlist_btn.click(
                generate_playlist,
                inputs=[playlist_type, num_tracks],
                outputs=playlist_output
            )
        
        with gr.Tab("📊 Analytics"):
            gr.Markdown("## 📈 Your Music Analytics")
            
            analytics_btn = gr.Button("📊 Get Detailed Analytics", variant="secondary")
            analytics_output = gr.Markdown(label="Music Analytics Dashboard")
            
            analytics_btn.click(get_user_analytics, outputs=analytics_output)
        
        with gr.Tab("🎧 Music Player"):
            gr.Markdown("## 🎵 Integrated Music Player")
            
            # Spotify Integration Notes
            gr.Markdown("""
            ### 🎵 **Spotify Integration Notes:**
            
            **For full song playback:**
            - Spotify links will open in your browser
            - Requires Spotify Premium for full tracks
            - Free users get 30-second previews
            - You must be logged into Spotify in your browser
            
            **Preview Integration:**
            - 30-second previews play directly in the interface
            - Available for most tracks (when provided by Spotify)
            - No Spotify account required for previews
            
            **If you get "No token provided" error:**
            - You need to be logged into your Spotify account in the browser
            - The app cannot directly authenticate users (limitation of Gradio interface)
            - For full playback, consider running this as a local web app with proper OAuth
            """)
            
            gr.Markdown("---")
            
            # Create audio components for each recommendation
            audio_players = []
            track_info_displays = []
            
            for i in range(5):  # Show 5 tracks with players
                with gr.Row():
                    with gr.Column(scale=2):
                        track_info = gr.Markdown(f"Track {i+1}: Load recommendations first")
                        track_info_displays.append(track_info)
                    with gr.Column(scale=1):
                        audio_player = gr.Audio(label=f"Track {i+1}", interactive=False)
                        audio_players.append(audio_player)
            
            refresh_player_btn = gr.Button("🔄 Load Current Recommendations in Player", variant="primary")
            
            # Flatten the outputs list for the interface
            all_outputs = []
            for i in range(5):
                all_outputs.extend([track_info_displays[i], audio_players[i]])
            
            refresh_player_btn.click(
                load_tracks_to_player,
                outputs=all_outputs
            )
        
        with gr.Tab("ℹ️ Enhanced Features"):
            gr.Markdown("""
            ## 🌟 Enhanced Features Overview
            
            ### 🎯 **Advanced AI Algorithms:**
            - **Hybrid Recommendation Engine**: Combines content-based, collaborative, and popularity-based filtering
            - **Context-Aware AI**: Recommendations adapt to your activity (workout, study, party, etc.)
            - **Dynamic Learning**: System improves recommendations based on your ratings and skips
            - **Smart Clustering**: Groups similar tracks and users for better recommendations
            
            ### 🎵 **Spotify Integration:**
            - **Direct Spotify Links**: Click to listen to full tracks on Spotify
            - **Preview Integration**: 30-second previews when available
            - **Real-time Metadata**: Latest track information, genres, and popularity scores
            - **Album Artwork**: Visual track information
            
            ### 🧠 **Intelligent Features:**
            - **Mood Detection**: Understands musical mood and energy levels
            - **Genre Intelligence**: Advanced genre-based recommendations
            - **Diversity Optimization**: Ensures varied recommendations to expand your taste
            - **Discovery Engine**: Introduces you to new artists and hidden gems
            
            ### 📊 **Advanced Analytics:**
            - **Listening Pattern Analysis**: Understand your music habits by time of day
            - **Taste Evolution Tracking**: See how your preferences change over time
            - **Recommendation Accuracy**: Track how well the AI knows your taste
            - **Discovery Metrics**: Measure how much new music you're finding
            
            ### 🎼 **Smart Playlists:**
            - **Context-Specific**: Workout, study, party, chill playlists
            - **Discovery Playlists**: Find new music tailored to your taste
            - **Throwback Collections**: Rediscover great music from the past
            - **Mood-Based**: Playlists that match your current emotional state
            
            ### 🔧 **Technical Improvements:**
            - **Memory Efficient**: Handles large datasets (1M+ tracks) smoothly
            - **Real-time Processing**: Fast recommendations with minimal latency
            - **Robust Error Handling**: Graceful fallbacks when features aren't available
            - **Scalable Architecture**: Designed for production deployment
            
            ### 🚀 **Getting Started:**
            1. **Setup**: Add your dataset path and optional Spotify credentials
            2. **Initialize**: Click "Initialize Enhanced System"
            3. **Explore**: Get context-aware recommendations
            4. **Rate**: Give ratings to improve AI understanding
            5. **Discover**: Use smart search and playlist generation
            6. **Analyze**: Check your music analytics dashboard
            
            ### 🛠️ **Spotify API Setup (Optional but Recommended):**
            1. Go to [Spotify Developer Dashboard](https://developer.spotify.com/dashboard)
            2. Create a new app
            3. Copy Client ID and Client Secret
            4. Add them in the Setup tab for full functionality
            
            ### 📝 **Tips for Best Results:**
            - Rate multiple tracks to help the AI learn your preferences
            - Try different contexts (workout, chill, etc.) for varied recommendations
            - Use the search feature to find specific tracks and rate them
            - Check analytics regularly to understand your music evolution
            - Generate different playlist types to discover new music
            """)
    
    return app
    
# Launch the enhanced application
if __name__ == "__main__":
    app = create_enhanced_gradio_interface()
    app.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a89006864c54400ee8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Spotify API initialized successfully!
Loading dataset from: /kaggle/input/spotify-12m-songs/tracks_features.csv
Loading dataset...
Available features: ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
Dataset loaded: 1204025 tracks with 16 features
Preprocessing features...
PCA applied: reduced from 16 to 8 dimensions
Enhanced preprocessing completed
Building enhanced similarity matrix...
Enhanced similarity matrix built
Could not get audio features for The Reason: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2w6fCVHQHCFh1DlEz3oUBx:
 None, reason: None
Could not get audio features for Till I'm 'Laid To Rest: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2gTurTIUUlCcsVxwspO3YG:
 None, reason: None
Could not get audio features for Stallin': http status: 403, code: -1 - https://api.spotify.com/v1/audio-featur